In [78]:
from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
#functions
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

from sklearn.svm import LinearSVC

from warnings import warn

import numpy as np
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets, feature_selection
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import label_ranking_average_precision_score

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import label_ranking_average_precision_score

from sklearn import tree

def multilabel_sample(y, size=1000, min_count=5, seed=None):
    """ Takes a matrix of binary labels `y` and returns
        the indices for a sample of size `size` if
        `size` > 1 or `size` * len(y) if size =< 1.
        The sample is guaranteed to have > `min_count` of
        each label.
    """
    try:
        if (np.unique(y).astype(int) != np.array([0, 1])).all():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('multilabel_sample only works with binary indicator matrices')

    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary.')

    if size <= 1:
        size = np.floor(y.shape[0] * size)

    if y.shape[1] * min_count > size:
        msg = "Size less than number of columns * min_count, returning {} items instead of {}."
        warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count

    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))

    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])

    sample_idxs = np.array([], dtype=choices.dtype)

    # first, guarantee > min_count of each label
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])


def multilabel_sample_dataframe(df, labels, size, min_count=5, seed=None):
    """ Takes a dataframe `df` and returns a sample of size `size` where all
        classes in the binary matrix `labels` are represented at
        least `min_count` times.
    """
    idxs = multilabel_sample(labels, size=size, min_count=min_count, seed=seed)
    return df.loc[idxs]


def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    """ Takes a features matrix `X` and a label matrix `Y` and
        returns (X_train, X_test, Y_train, Y_test) where all
        classes in Y are represented at least `min_count` times.
    """
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])

    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])






In [45]:
df = pd.read_csv('TK2010_revised.csv', sep=',' ,encoding= 'latin1')
df = df[['gemeente','Partij']]
df2 = pd.read_csv('2010_all.csv',sep=',')
df2 = df2.rename(columns={"Regio's": "gemeente"})


df3 = pd.merge(df,df2, on='gemeente')


NUMBER = list(df3.columns[3:])


y = df3['Partij'].values
X = df3[NUMBER].fillna(-1000).values


In [89]:
ff = pd.read_csv('TK2017_revised.csv', sep=';' ,encoding= 'latin1')
ff = ff[['gemeente','Partij']]
ff2 = pd.read_csv('2017_all.csv',sep=',')
ff2 = ff2.rename(columns={"Regio's": "gemeente"})


ff3 = pd.merge(ff,ff2, on='gemeente')


NUMBER = list(ff3.columns[3:])


y2 = ff3['Partij'].values
X2 = ff3[NUMBER].fillna(-1000).values





In [67]:
from sklearn.svm import LinearSVC

from sklearn.feature_selection import SelectFromModel


print(X.shape)

lsvc = LinearSVC(C=0.001, penalty="l2", dual=False).fit(X,y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)


(1656, 271)
(1656, 91)


In [91]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01,penalty="l2"))),
  ('classification', tree.DecisionTreeClassifier(max_depth=6))
])
clf.fit(X, y)

# Predict the labels of the test data: y_pred
y_pred = clf.predict(X2)

print(len(y_pred))

# Generate the confusion matrix and classification report
#print(confusion_matrix(y_test, y_pred))
print(classification_report(y, y_pred))
print(label_ranking_average_precision_score(y, y_pred)) 


1875


ValueError: Found input variables with inconsistent numbers of samples: [1656, 1875]